## NOTE: 
Since my Office 365's lincense is expired, I have to use Jupyter Notebook to demonstrate all of the data processing steps and visualization. As usual, I would prefer Excel dashboard for better visualization!


In [58]:
import pandas as pd
import numpy as np

def highlight1(data, color='yellow'):
    color = 'yellow' if data > 0.2 else ''
    return 'background-color: {}'.format(color)

In [59]:
df = pd.read_csv("tiki_test.csv")
# Review the first 5 rows of original dataset
df.head()

,Seller ID,Seller's Vertical,Sign-up Time,Activation Time,1st Listing,1st Salable,1st Transaction
0,1,Electronic,9/20/2017,2/2/2018,2/7/2018,10/6/2018,10/10/2018
1,2,BBFF,8/19/2017,5/24/2018,8/13/2018,8/14/2018,9/2/2018
2,3,BBFF,12/4/2017,5/4/2018,10/6/2018,10/10/2018,10/12/2018
3,4,BBFF,10/16/2017,4/23/2018,7/4/2018,7/10/2018,7/10/2018
4,5,BBFF,12/13/2017,7/25/2018,8/30/2018,8/31/2018,9/14/2018


In [60]:
# Conver to string to datetime
lst_step = df.columns.values[2:7]
df[lst_step] = df[lst_step].astype('datetime64[ns]')

lst_gap = ["Step 1 to 2 (months)", "Step 2 to 3 (months)", "Step 3 to 4 (months)", "Step 4 to 5 (months)", "Step 1 to 5 (months)"]

# Create new cols
for i in range(len(lst_gap)):
    if i < 4:
        df[lst_gap[i]] = round((df[lst_step[i+1]] - df[lst_step[i]])/np.timedelta64(1,'M'))
    else:
        df[lst_gap[i]] = round((df[lst_step[i]] - df[lst_step[0]])/np.timedelta64(1,'M'))


### Transform original dataset to a new one.
Step 1 to 2: From Sign-up Time To Activation Time.  
Step 2 to 3: From Activation Time to 1st Listing.  
Step 3 to 4: From 1st Listing to 1st Salable.  
Step 4 to 5: From 1st Salable To 1st Transaction.  
Step 1 to 5: From Sign-up Time To 1st Transaction.
### To investigate the length of time during each step, the new dataset is as follow:

In [61]:
# Create a new list of columns
lst_all = df.columns.values
lst_new = [x for x in lst_all if x not in lst_step]
# New table
df_new = df[lst_new]
# Review the first 5 rows of new dataset
df_new.head()

,Seller ID,Seller's Vertical,Step 1 to 2 (months),Step 2 to 3 (months),Step 3 to 4 (months),Step 4 to 5 (months),Step 1 to 5 (months)
0,1,Electronic,4.0,0.0,8.0,0.0,13.0
1,2,BBFF,9.0,3.0,0.0,1.0,12.0
2,3,BBFF,5.0,5.0,0.0,0.0,10.0
3,4,BBFF,6.0,2.0,0.0,0.0,9.0
4,5,BBFF,7.0,1.0,0.0,0.0,9.0


In [62]:
# Create a pivot table
pivot1 = pd.pivot_table(df_new, index=["Seller's Vertical"], values=['Seller ID'], aggfunc=len, margins=True, margins_name="Total").rename(columns={"Seller ID": "# Sellers"})
# Add '% Distribution' column
f = lambda x: x /pivot1.loc['Total', "# Sellers"]
pivot1['% Distribution'] = f(pivot1['# Sellers'])

# Save a new version of table
pivot1 = pivot1.style.applymap(highlight1, subset=pd.IndexSlice['BBFF': 'Lifestyle', ['% Distribution']]).format({'% Distribution': "{:.0%}"})
# Review pivot table
pivot1

,# Sellers,% Distribution
Seller's Vertical,,
BBFF,1130,53%
Book,130,6%
Digital Service,122,6%
Electronic,262,12%
LifeStyle,501,23%
Total,2145,100%


### First note:
The majority of sellers had main categories in BBFF (53%) and Lifestyle (23%). Although Book is a famous category of Tiki, the number of sellers was only around 6% of the platform.

In [63]:
df['Sign-up Y_m'] = pd.to_datetime(df['Sign-up Time']).dt.to_period('M')
df2 = df[["Seller ID", "Seller's Vertical", "Sign-up Y_m"]]
# df2.head()

In [64]:
# Create a pivot table to see the # sellers in each Sign-up Year_Month
pivot_signup = pd.pivot_table(df2, index=["Sign-up Y_m"], columns=["Seller's Vertical"], values=['Seller ID'], aggfunc=len, margins=True, margins_name="Total", fill_value=0).rename(columns={"Seller ID": "# Sellers"})
# Review
pivot_signup

# Sellers                                                
Seller's Vertical      BBFF Book Digital Service Electronic LifeStyle Total
Sign-up Y_m                                                                
2017-08                   1    0               0          0         0     1
2017-09                   0    0               0          2         0     2
2017-10                   2    0               0          0         1     3
2017-11                   2    0               0          0         1     3
2017-12                   3    0               0          0         0     3
2018-01                  30    2               7          5        11    55
2018-02                  34    3               1          7        15    60
2018-03                 129   13              22         40        50   254
2018-04                 126   20              27         47        77   297
2018-05                 184   17              15         40        74   330
2018-06                 163   18              11         22        84   298
2018-07                 142   16              10         28        55   251
2018-08                 189   16               9         39        80   333
2018-09                 112   21              17         26        44   220
2018-10                  13    4               3          6         9    35
Total                  1130  130             122        262       501  2145

### Second note:
There were nearly no sellers signing up from 2017-08 to 2017-12 (this result is weird!). Until 2018-01 and 2018-02, those numbers slightly increased but mainly at BBFF and Lifestyle.
  
However, there was an incredible jump in number of registers during the course of 7 consecutive months from 2018-03, accounting for 5 - 10 times higher than the previous months. Perhaps, Quarter 2 and Quarter 3 of the year were the main selling season of Tiki; therefore, most of sellers joining in this time.



In [65]:
# Create a pivot table for the step transition
pivot2 = pd.pivot_table(df_new, index=["Step 1 to 2 (months)"], columns=["Seller's Vertical"], values=['Seller ID'], aggfunc=len, margins=True, fill_value=0).rename(columns={"Seller ID": "# Sellers"})

pivot3 = pd.pivot_table(df_new, index=["Step 2 to 3 (months)"], columns=["Seller's Vertical"], values=['Seller ID'], aggfunc=len, margins=True, fill_value=0).rename(columns={"Seller ID": "# Sellers"})

pivot4 = pd.pivot_table(df_new, index=["Step 3 to 4 (months)"], columns=["Seller's Vertical"], values=['Seller ID'], aggfunc=len, margins=True, fill_value=0).rename(columns={"Seller ID": "# Sellers"})

pivot5 = pd.pivot_table(df_new, index=["Step 4 to 5 (months)"], columns=["Seller's Vertical"], values=['Seller ID'], aggfunc=len, margins=True, fill_value=0).rename(columns={"Seller ID": "# Sellers"})

pivot6 = pd.pivot_table(df_new, index=["Step 1 to 5 (months)"], columns=["Seller's Vertical"], values=['Seller ID'], aggfunc=len, margins=True, fill_value=0).rename(columns={"Seller ID": "# Sellers"})

lst1_pv = [pivot2, pivot3, pivot4, pivot5, pivot6]
for i in lst1_pv:
    print(i,"\n")

# Sellers                                                
Seller's Vertical         BBFF Book Digital Service Electronic LifeStyle   All
Step 1 to 2 (months)                                                          
0.0                        787   87              66        201       337  1478
1.0                        264   33              39         41       121   498
2.0                         47    6               9         13        29   104
3.0                         18    2               3          3         6    32
4.0                          7    2               4          3         4    20
5.0                          3    0               1          1         3     8
6.0                          2    0               0          0         1     3
7.0                          1    0               0          0         0     1
9.0                          1    0               0          0         0     1
All                       1130  130             122        262       501 

### Third note:  
Looking at the transition between each step, the lenghth of time was mainly about less 2 months to complete. Eventually, it would take around 1 to 2 months for a new seller to land the 1st transaction. For BBFF and Lifestyle categories, some sellers tooks more than 3 month to see the order.

In [66]:
df['1st Transaction Y_m'] = pd.to_datetime(df['1st Transaction']).dt.to_period('M')
df3 = df[["Seller ID", "Seller's Vertical", "1st Transaction Y_m"]]
df3.head()

,Seller ID,Seller's Vertical,1st Transaction Y_m
0,1,Electronic,2018-10
1,2,BBFF,2018-09
2,3,BBFF,2018-10
3,4,BBFF,2018-07
4,5,BBFF,2018-09


In [67]:
# Create a pivot table to see the # sellers when landing the 1st Transaction
pivot_transaction = pd.pivot_table(df3, index=["1st Transaction Y_m"], columns=["Seller's Vertical"], values=['Seller ID'], aggfunc=len, margins=True, margins_name="Total", fill_value=0).rename(columns={"Seller ID": "# Sellers"})
# Review
pivot_transaction

# Sellers                                                
Seller's Vertical        BBFF Book Digital Service Electronic LifeStyle Total
1st Transaction Y_m                                                          
2018-02                     4    0               0          0         0     4
2018-03                    25    2               4          6        12    49
2018-04                    74    8               8         22        23   135
2018-05                    87   14              15         30        47   193
2018-06                   143   18              16         35        71   283
2018-07                   177   26              19         30        72   324
2018-08                   198   19              19         54       104   394
2018-09                   232   28              24         41        93   418
2018-10                   190   15              17         44        79   345
Total                    1130  130             122        262       501  2145

### Fourth note:
Like the result indicated in Second note, the 1st transactions were also generated in same time of the increase in registers among Quarter 2 and especially Quarter 3 of the year.

In [68]:
df4 = df[["Seller ID", "Seller's Vertical", "Sign-up Y_m", "Step 1 to 5 (months)"]]
df4.head()

,Seller ID,Seller's Vertical,Sign-up Y_m,Step 1 to 5 (months)
0,1,Electronic,2017-09,13.0
1,2,BBFF,2017-08,12.0
2,3,BBFF,2017-12,10.0
3,4,BBFF,2017-10,9.0
4,5,BBFF,2017-12,9.0


In [69]:
# Create a pivot table to see the # sellers when landing the 1st Transaction
pivot_con1 = pd.pivot_table(df4, index=["Seller's Vertical", "Sign-up Y_m"], columns=["Step 1 to 5 (months)"], values=['Seller ID'], aggfunc=len, fill_value=0).rename(columns={"Seller ID": "# Sellers"})
# Highlight values being greater than 0
def highlight2(data, color='yellow'):
    color = 'yellow' if data > 0 else ''
    return 'background-color: {}'.format(color)
# Review filtered table
pivot_con1.style.applymap(highlight2)

### Fifth note:
The sellers who registered in Quarter 2 and 3 of 2018 were more likely to land 1st Transaction within the next 3 months.

### Conclusion:
1. Majority of sellers' main categories were allocated in BBFF and Lifestyle.  
2. Every steps of selling registration took less than 2 months to complete. In total, it could take up to 3 months from signing up to get the 1st transaction.  
3. Sellers signed in Quarter 2 and 3 of the year were more likely to land the 1st transaction within the next 3 months. 

### Recommendation:
Taking up to 3 months to see the 1st transaction is fairly long for a seller. He/she cannot be patient enough to stay on the platform. It would be better if Tiki can reduce the transitions from step 1 to 4 within 1 month so that seller's products can be launched and boosted by campaign package in order to realize the 1st transaction. 